In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    DDD_Train_Decile0 = pd.read_pickle("DDD_Train_Decile0.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile0.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile0, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 0')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

2264/2264 [==============================] - 1s 620us/step - loss: 0.6272 - acc: 0.6568 - val_loss: 0.6059 - val_acc: 0.6808
Epoch 35/75
2264/2264 [==============================] - 1s 601us/step - loss: 0.6250 - acc: 0.6466 - val_loss: 0.6059 - val_acc: 0.6825
Epoch 36/75
2264/2264 [==============================] - 1s 552us/step - loss: 0.6283 - acc: 0.6550 - val_loss: 0.6058 - val_acc: 0.6825
Epoch 37/75
2264/2264 [==============================] - 1s 566us/step - loss: 0.6258 - acc: 0.6612 - val_loss: 0.6058 - val_acc: 0.6825
Epoch 38/75
2264/2264 [==============================] - 2s 677us/step - loss: 0.6262 - acc: 0.6542 - val_loss: 0.6059 - val_acc: 0.6825
Epoch 39/75
2264/2264 [==============================] - 1s 621us/step - loss: 0.6212 - acc: 0.6506 - val_loss: 0.6058 - val_acc: 0.6825
Epoch 40/75
2264/2264 [==============================] - 1s 595us/step - loss: 0.6263 - acc: 0.6590 - val_loss: 0.6059 - val_acc: 0.6808
Epoch 41/75
2264/2264 [==============================

Epoch 47/125
2264/2264 [==============================] - 1s 610us/step - loss: 0.6244 - acc: 0.6572 - val_loss: 0.6059 - val_acc: 0.6720
Epoch 48/125
2264/2264 [==============================] - 1s 638us/step - loss: 0.6247 - acc: 0.6599 - val_loss: 0.6059 - val_acc: 0.6720
Epoch 49/125
2264/2264 [==============================] - 1s 636us/step - loss: 0.6228 - acc: 0.6546 - val_loss: 0.6059 - val_acc: 0.6702
Epoch 50/125
2264/2264 [==============================] - 1s 580us/step - loss: 0.6236 - acc: 0.6572 - val_loss: 0.6060 - val_acc: 0.6702
Epoch 51/125
2264/2264 [==============================] - 1s 562us/step - loss: 0.6221 - acc: 0.6564 - val_loss: 0.6060 - val_acc: 0.6702
Epoch 52/125
2264/2264 [==============================] - 1s 590us/step - loss: 0.6251 - acc: 0.6656 - val_loss: 0.6060 - val_acc: 0.6702
Epoch 53/125
2264/2264 [==============================] - 1s 615us/step - loss: 0.6187 - acc: 0.6599 - val_loss: 0.6060 - val_acc: 0.6667
Epoch 54/125
2264/2264 [==========

2264/2264 [==============================] - 1s 561us/step - loss: 0.6229 - acc: 0.6511 - val_loss: 0.6061 - val_acc: 0.6667
Epoch 107/125
2264/2264 [==============================] - 1s 568us/step - loss: 0.6203 - acc: 0.6599 - val_loss: 0.6061 - val_acc: 0.6702
Epoch 108/125
2264/2264 [==============================] - 1s 579us/step - loss: 0.6196 - acc: 0.6643 - val_loss: 0.6061 - val_acc: 0.6649
Epoch 109/125
2264/2264 [==============================] - 1s 558us/step - loss: 0.6183 - acc: 0.6572 - val_loss: 0.6061 - val_acc: 0.6649
Epoch 110/125
2264/2264 [==============================] - 1s 576us/step - loss: 0.6245 - acc: 0.6692 - val_loss: 0.6061 - val_acc: 0.6667
Epoch 111/125
2264/2264 [==============================] - 1s 601us/step - loss: 0.6199 - acc: 0.6564 - val_loss: 0.6061 - val_acc: 0.6649
Epoch 112/125
2264/2264 [==============================] - 1s 579us/step - loss: 0.6247 - acc: 0.6564 - val_loss: 0.6061 - val_acc: 0.6667
Epoch 113/125
2264/2264 [================

2264/2264 [==============================] - 1s 558us/step - loss: 0.6247 - acc: 0.6493 - val_loss: 0.6045 - val_acc: 0.6808
Epoch 40/100
2264/2264 [==============================] - 1s 608us/step - loss: 0.6290 - acc: 0.6515 - val_loss: 0.6044 - val_acc: 0.6808
Epoch 41/100
2264/2264 [==============================] - 1s 607us/step - loss: 0.6251 - acc: 0.6564 - val_loss: 0.6044 - val_acc: 0.6790
Epoch 42/100
2264/2264 [==============================] - 1s 552us/step - loss: 0.6262 - acc: 0.6502 - val_loss: 0.6044 - val_acc: 0.6808
Epoch 43/100
2264/2264 [==============================] - 1s 571us/step - loss: 0.6246 - acc: 0.6528 - val_loss: 0.6044 - val_acc: 0.6808
Epoch 44/100
2264/2264 [==============================] - 1s 557us/step - loss: 0.6245 - acc: 0.6555 - val_loss: 0.6044 - val_acc: 0.6825
Epoch 45/100
2264/2264 [==============================] - 1s 559us/step - loss: 0.6301 - acc: 0.6595 - val_loss: 0.6044 - val_acc: 0.6825
Epoch 46/100
2264/2264 [=======================

...

2264/2264 [==============================] - 1s 630us/step - loss: 0.6518 - acc: 0.6400 - val_loss: 0.6364 - val_acc: 0.6684
Epoch 6/125
2264/2264 [==============================] - 1s 657us/step - loss: 0.6463 - acc: 0.6383 - val_loss: 0.6268 - val_acc: 0.6667
Epoch 7/125
2264/2264 [==============================] - 1s 607us/step - loss: 0.6332 - acc: 0.6436 - val_loss: 0.6177 - val_acc: 0.6420
Epoch 8/125
2264/2264 [==============================] - 1s 644us/step - loss: 0.6329 - acc: 0.6418 - val_loss: 0.6134 - val_acc: 0.6420
Epoch 9/125
2264/2264 [==============================] - 1s 586us/step - loss: 0.6292 - acc: 0.6440 - val_loss: 0.6107 - val_acc: 0.6420
Epoch 10/125
2264/2264 [==============================] - 1s 540us/step - loss: 0.6288 - acc: 0.6449 - val_loss: 0.6091 - val_acc: 0.6420
Epoch 11/125
2264/2264 [==============================] - 1s 538us/step - loss: 0.6286 - acc: 0.6449 - val_loss: 0.6081 - val_acc: 0.6420
Epoch 12/125
2264/2264 [===========================

2264/2264 [==============================] - 1s 413us/step - loss: 0.6261 - acc: 0.6489 - val_loss: 0.6067 - val_acc: 0.6808
Epoch 65/125
2264/2264 [==============================] - 1s 409us/step - loss: 0.6247 - acc: 0.6528 - val_loss: 0.6067 - val_acc: 0.6808
Epoch 66/125
2264/2264 [==============================] - 1s 448us/step - loss: 0.6278 - acc: 0.6519 - val_loss: 0.6067 - val_acc: 0.6825
Epoch 67/125
2264/2264 [==============================] - 1s 413us/step - loss: 0.6236 - acc: 0.6537 - val_loss: 0.6067 - val_acc: 0.6843
Epoch 68/125
2264/2264 [==============================] - 1s 412us/step - loss: 0.6294 - acc: 0.6519 - val_loss: 0.6067 - val_acc: 0.6843
Epoch 69/125
2264/2264 [==============================] - 1s 414us/step - loss: 0.6200 - acc: 0.6546 - val_loss: 0.6067 - val_acc: 0.6843
Epoch 70/125
2264/2264 [==============================] - 1s 432us/step - loss: 0.6259 - acc: 0.6475 - val_loss: 0.6068 - val_acc: 0.6843
Epoch 71/125
2264/2264 [=======================

2264/2264 [==============================] - 1s 621us/step - loss: 0.6298 - acc: 0.6524 - val_loss: 0.6067 - val_acc: 0.6790
Epoch 124/125
2264/2264 [==============================] - 1s 610us/step - loss: 0.6245 - acc: 0.6489 - val_loss: 0.6067 - val_acc: 0.6790
Epoch 125/125
2264/2264 [==============================] - 1s 619us/step - loss: 0.6277 - acc: 0.6559 - val_loss: 0.6067 - val_acc: 0.6790
Test accuracy: 0.679012346205
Train on 2264 samples, validate on 567 samples
Epoch 1/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.6589 - acc: 0.6206 - val_loss: 0.6151 - val_acc: 0.6455
Epoch 2/100
2264/2264 [==============================] - 1s 433us/step - loss: 0.6335 - acc: 0.6427 - val_loss: 0.6065 - val_acc: 0.6455
Epoch 3/100
2264/2264 [==============================] - 1s 421us/step - loss: 0.6301 - acc: 0.6453 - val_loss: 0.6038 - val_acc: 0.6702
Epoch 4/100
2264/2264 [==============================] - 1s 428us/step - loss: 0.6280 - acc: 0.6475 - val_loss:

2264/2264 [==============================] - 1s 601us/step - loss: 0.6228 - acc: 0.6542 - val_loss: 0.6038 - val_acc: 0.6702
Epoch 56/100
2264/2264 [==============================] - 1s 604us/step - loss: 0.6277 - acc: 0.6511 - val_loss: 0.6039 - val_acc: 0.6702
Epoch 57/100
2264/2264 [==============================] - 1s 659us/step - loss: 0.6229 - acc: 0.6537 - val_loss: 0.6039 - val_acc: 0.6702
Epoch 58/100
2264/2264 [==============================] - 1s 607us/step - loss: 0.6224 - acc: 0.6608 - val_loss: 0.6039 - val_acc: 0.6702
Epoch 59/100
2264/2264 [==============================] - 1s 563us/step - loss: 0.6218 - acc: 0.6586 - val_loss: 0.6039 - val_acc: 0.6702
Epoch 60/100
2264/2264 [==============================] - 1s 579us/step - loss: 0.6230 - acc: 0.6599 - val_loss: 0.6039 - val_acc: 0.6702
Epoch 61/100
2264/2264 [==============================] - 1s 596us/step - loss: 0.6215 - acc: 0.6515 - val_loss: 0.6039 - val_acc: 0.6702
Epoch 62/100
2264/2264 [=======================

2264/2264 [==============================] - 1s 621us/step - loss: 0.6218 - acc: 0.6643 - val_loss: 0.6074 - val_acc: 0.6843
Epoch 14/50
2264/2264 [==============================] - 2s 676us/step - loss: 0.6226 - acc: 0.6533 - val_loss: 0.6073 - val_acc: 0.6825
Epoch 15/50
2264/2264 [==============================] - 1s 615us/step - loss: 0.6200 - acc: 0.6595 - val_loss: 0.6073 - val_acc: 0.6808

Epoch 00015: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 16/50
2264/2264 [==============================] - 1s 535us/step - loss: 0.6189 - acc: 0.6625 - val_loss: 0.6073 - val_acc: 0.6808
Epoch 17/50
2264/2264 [==============================] - 1s 607us/step - loss: 0.6239 - acc: 0.6590 - val_loss: 0.6073 - val_acc: 0.6808
Epoch 18/50
2264/2264 [==============================] - 1s 535us/step - loss: 0.6231 - acc: 0.6559 - val_loss: 0.6073 - val_acc: 0.6808
Epoch 19/50
2264/2264 [==============================] - 1s 566us/step - loss: 0.6252 - acc: 0.6528 - val_loss: 0.6073 - val_

In [ ]:
0.86434